### 3. Consider the alphabet prediction problem discussed in the class.
### (a) Generate 500 alphabets from the set {b, c, d, f} uniformly at random. Then convert each alphabet as follows:
### b → ba, c → ce, d → di, f → fo.
### We thus have a set of features X1, X2 . . . , X1000 ∈ {b, c, d, f, a, e, i, o}.
### Print X1, X2, . . . , X20. The task is to predict the next alphabet. So
### fix the responses Yi = Xi+1 with Y1000 generated randomly.

In [3]:
import random

alphabet_set = ['b', 'c', 'd', 'f', 'a', 'e', 'i', 'o']
features_list = []
for i in range(500):
    char = random.choice(['b', 'c', 'd', 'f'])
    if char == 'b':
        features_list.extend(['b', 'a'])
    elif char == 'c':
        features_list.extend(['c', 'e'])
    elif char == 'd':
        features_list.extend(['d', 'i'])
    else:
        features_list.extend(['f', 'o'])

# Print the first 20 features
print(features_list[:20],len(features_list))

# Generate the corresponding responses
Y_list = features_list[1:] + [random.choice(alphabet_set) for _ in range(1)]
print(Y_list[:20])
Y = [alphabet_set.index(char) for char in Y_list]

# Print the first 20 responses
# print(Y[:20])


['f', 'o', 'f', 'o', 'd', 'i', 'f', 'o', 'b', 'a', 'b', 'a', 'f', 'o', 'c', 'e', 'd', 'i', 'f', 'o'] 1000
['o', 'f', 'o', 'd', 'i', 'f', 'o', 'b', 'a', 'b', 'a', 'f', 'o', 'c', 'e', 'd', 'i', 'f', 'o', 'd']


### (b) Construct a recurrent neural network using a SimpleRNN layer and
### a Dense layer from tensorflow.keras.layers. Choose the number of
### nodes and the activation function so as to minimize the cross-entropy
### loss function. Print the fraction of misclassified consonants and mis-classified vowels.

In [5]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

# Define the alphabet set
alphabet_set = ['b', 'c', 'd', 'f', 'a', 'e', 'i', 'o']

# Convert the features and responses to numerical arrays
X = np.zeros((500, 8))
Y = np.zeros(500)
for i, feature in enumerate(features_list) :
    if i>= 500:
        break
    if (feature in alphabet_set) &(i<500) :
        X[i, alphabet_set.index(feature)] = 1
    elif(feature == 'a') & (i<500) :
        X[i, len(alphabet_set)] = 1
    elif (feature == 'e') & (i<500):
        X[i, len(alphabet_set) + 1] = 1
    elif (feature == 'i') & (i<500):
        X[i, len(alphabet_set) + 2] = 1
    elif (feature == 'o') & (i<500):
        X[i, len(alphabet_set) + 3] = 1
    Y[i] = alphabet_set.index(Y_list[i])

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(32, input_shape=(1, 8), activation='tanh'))
model.add(Dense(len(alphabet_set), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X.reshape(-1, 1, 8), Y, epochs=50, batch_size=16, verbose=0)

# Use the model to make predictions
# y_pred = model.predict_classes(X.reshape(-1, 1, 8))
y_pred_prob = model.predict(X.reshape(-1, 1, 8))
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = Y.astype(int)

# Calculate the fraction of misclassified consonants and vowels
consonant_indices = [alphabet_set.index(c) for c in ['b', 'c', 'd', 'f']]
consonant_errors = np.sum(np.logical_and(y_pred != y_true, np.isin(y_true, consonant_indices)))
consonant_total = np.sum(np.isin(y_true, consonant_indices))
consonant_error_fraction = consonant_errors / consonant_total

vowel_indices = [alphabet_set.index(v) for v in ['a', 'e', 'i', 'o']]
vowel_errors = np.sum(np.logical_and(y_pred != y_true, np.isin(y_true, vowel_indices)))
vowel_total = np.sum(np.isin(y_true, vowel_indices))
vowel_error_fraction = vowel_errors / vowel_total

print("Fraction of misclassified consonants:", consonant_error_fraction)
print("Fraction of misclassified vowels:", vowel_error_fraction)


16/16 [==============================] - 0s 2ms/step
Fraction of misclassified consonants: 0.684
Fraction of misclassified vowels: 0.0
